Initialization pof the IPC Communication 
1. Open OPENLCA, activate the relevent database
2. Tools > Developper Tools > IPC Server > run the server with the correct port

In [1]:
import olca_ipc as ipc
import olca_schema as o
import pandas as pd
import numpy as np
import json

from typing import Callable
###########################################################
client = ipc.Client(8080)

Technosphere matrice for 4 process with the 4 process 


In [2]:
technosphere = pd.DataFrame(
    data=[
        [1.0, -50.0, -1.0, 0.0],
        [-0.01, 1.0, -1.0, 0.0],
        [0.0, 0.0, 1.0, -1.0],
        [0.0, 0.0, 0.0, 100],
    ],
    columns=[
        "electricity production",
        "aluminium production",
        "aluminium foil production",
        "sandwitch package production",
    ],
    index=[
        "electricity [MJ]",
        "aluminium [kg]",
        "aluminium foil [kg]",
        "sandwitch package [Item(s)]",
    ],
)
print(technosphere)

                             electricity production  aluminium production  \
electricity [MJ]                               1.00                 -50.0   
aluminium [kg]                                -0.01                   1.0   
aluminium foil [kg]                            0.00                   0.0   
sandwitch package [Item(s)]                    0.00                   0.0   

                             aluminium foil production  \
electricity [MJ]                                  -1.0   
aluminium [kg]                                    -1.0   
aluminium foil [kg]                                1.0   
sandwitch package [Item(s)]                        0.0   

                             sandwitch package production  
electricity [MJ]                                      0.0  
aluminium [kg]                                        0.0  
aluminium foil [kg]                                  -1.0  
sandwitch package [Item(s)]                         100.0  


technosphere to biosphere 

In [3]:
interventions = pd.DataFrame(
    data=[
        [0.0, -5.0, 0.0, 0.0],
        [-0.5, 0.0, 0.0, 0.0],
        [3.0, 0.0, 0.0, 0.0],
        [2.0, 10.0, 0.0, 1.0],
    ],
    columns=technosphere.columns,
    index=[
        "bauxite [kg]",
        "crude oil [kg]",
        "CO2 [kg]",
        "solid waste [kg]",
    ],
)
print(interventions)

                  electricity production  aluminium production  \
bauxite [kg]                         0.0                  -5.0   
crude oil [kg]                      -0.5                   0.0   
CO2 [kg]                             3.0                   0.0   
solid waste [kg]                     2.0                  10.0   

                  aluminium foil production  sandwitch package production  
bauxite [kg]                            0.0                           0.0  
crude oil [kg]                          0.0                           0.0  
CO2 [kg]                                0.0                           0.0  
solid waste [kg]                        0.0                           1.0  


In [4]:
f = [
    0.0,
    0.0,
    0.0,
    10,
]
s = np.linalg.solve(technosphere.to_numpy(), f)
g = interventions.to_numpy() @ s
print(pd.DataFrame(g, index=interventions.index))

                      0
bauxite [kg]      -1.01
crude oil [kg]    -5.10
CO2 [kg]          30.60
solid waste [kg]  22.52


In [6]:
mass_units = o.new_unit_group("Mass units", "kg")
energy_units = o.new_unit_group("Energy units", "MJ")
counting_units = o.new_unit_group("Counting units", "Item(s)")
mass = o.new_flow_property("Mass", mass_units)
energy = o.new_flow_property("Energy", energy_units)
count = o.new_flow_property("Number of items", counting_units)

client.put_all(
    mass_units,
    energy_units,
    counting_units,
    mass,
    energy,
    count,
)
print(client.get(o.FlowProperty, name="Mass").to_json())

{
  "@type": "FlowProperty",
  "@id": "93a60a56-a3c8-11da-a746-0800200b9a66",
  "category": "Technical flow properties",
  "description": "",
  "flowPropertyType": "PHYSICAL_QUANTITY",
  "lastChange": "2023-11-20T07:53:54.682948Z",
  "name": "Mass",
  "unitGroup": {
    "@type": "UnitGroup",
    "@id": "93a60a57-a4c8-11da-a746-0800200c9a66",
    "category": "Technical unit groups",
    "name": "Units of mass"
  },
  "version": "00.00.000"
}


In [16]:
def create_flow(
    row_label: str, fn: Callable[[str, o.FlowProperty], o.Flow]
) -> o.Flow:
    parts = row_label.split("[")
    name = parts[0].strip()
    unit = parts[1][0:-1].strip()
    match unit:
        case "kg":
            prop = mass
        case "MJ":
            prop = energy
        case "Item(s)":
            prop = count
    flow = fn(name, prop)
    client.put(flow)
    return flow


tech_flows = [create_flow(label, o.new_product) for label in technosphere.index]
envi_flows = [
    create_flow(label, o.new_elementary_flow) for label in interventions.index
]

In [17]:
def create_process(index: int, name: str) -> o.Process:
    process = o.new_process(name)

    def exchange(flow: o.Flow, value: float) -> o.Exchange | None:
        if value == 0:
            return None
        if value < 0:
            return o.new_input(process, flow, abs(value))
        else:
            return o.new_output(process, flow, value)

    for (i, tech_flow) in enumerate(tech_flows):
        value = technosphere.iat[i, index]
        e = exchange(tech_flow, value)
        if e and i == index:
            e.is_quantitative_reference = True

    for (i, envi_flow) in enumerate(envi_flows):
        value = interventions.iat[i, index]
        exchange(envi_flow, value)

    client.put(process)
    return process


processes = [
    create_process(index, name)
    for (index, name) in enumerate(technosphere.columns)
]

Calculation setup 1

In [18]:
setup = o.CalculationSetup(
    target=o.Ref(ref_type=o.RefType.Process, id=processes[3].id),
    unit=count.unit_group.ref_unit,  # "Item(s)"
    amount=10,
)
result = client.calculate(setup)
result.wait_until_ready()

inventory = result.get_total_flows()
print(
    pd.DataFrame(
        data=[
            (
                i.envi_flow.flow.name,
                i.envi_flow.is_input,
                i.amount,
                i.envi_flow.flow.ref_unit,
            )
            for i in inventory
        ],
        columns=["Flow", "Is input?", "Amount", "Unit"],
    )
)

result.dispose()

          Flow  Is input?  Amount Unit
0    crude oil       True    0.00   kg
1  solid waste      False    0.00   kg
2          CO2      False    0.00   kg
3      bauxite       True    0.00   kg
4  solid waste      False   22.52   kg
5          CO2      False   30.60   kg
6    crude oil       True    5.10   kg
7      bauxite       True    1.01   kg


In [21]:
models = client.get_descriptors(o.ProductSystem)
for model in models:
    print(f"{model.name} :: {model.id}")
model = models[0]

methods = client.get_descriptors(o.ImpactMethod)
for method in methods:
    print(f"{method.name} :: {method.id}")
method = methods[0]

sandwitch package production :: 87257a67-2f84-4486-9528-22591cd3a803
aluminium foil production :: 7fb896e0-082b-4593-b13d-71a3fee71059
IMPACTWorld+ (Default_Recommended_Midpoint 1.29) :: 8280f30d-2cdd-3451-b0d2-2e439f836b1e
IMPACTWorld+ (Default_Recommended_Damage 1.47) :: ad46ddc3-1972-3261-bcce-5223501ad816
IMPACTWorld+ (Continental Latin America_Damage 1.431) :: 563f0041-cb44-3e57-9409-d1ced60b8a2d
IMPACTWorld+ (Continental Latin America_Midpoint 1.251) :: c9c45b61-474f-3344-a19d-51942842dac4


Calculation setup 2


In [22]:
setup = o.CalculationSetup(
    target=model,
    impact_method=method,
)
print(json.dumps(setup.to_dict(), indent=2))
##### Calculation ####
result = client.calculate(setup)
result.wait_until_ready()
impacts = result.get_total_impacts()
for i in impacts:
    assert i.impact_category
    print(f"{i.impact_category.name} {i.amount} {i.impact_category.ref_unit}")
result.dispose()



{
  "impactMethod": {
    "@type": "ImpactMethod",
    "@id": "8280f30d-2cdd-3451-b0d2-2e439f836b1e",
    "name": "IMPACTWorld+ (Default_Recommended_Midpoint 1.29)"
  },
  "target": {
    "@type": "ProductSystem",
    "@id": "87257a67-2f84-4486-9528-22591cd3a803",
    "name": "sandwitch package production"
  }
}
Climate change, long term 306.0 kg CO2 eq (long)
Photochemical oxidant formation 0.0 kg NMVOC eq
Freshwater acidification 0.0 kg SO2 eq
Mineral resources use 0.0 kg deprived
Climate change, short term 306.0 kg CO2 eq (short)
Land transformation, biodiversity 0.0 m2 arable land eq
Land occupation, biodiversity 0.0 m2 arable land eq .yr
Marine eutrophication 0.0 kg N N-lim eq
Human toxicity non cancer 0.0 CTUh
Particulate matter formation 0.0 kg PM2.5 eq
Ozone Layer Depletion 0.0 kg  CFC-11 eq
Freshwater ecotoxicity 0.0 CTUe
Terrestrial acidification 0.0 kg SO2 eq
Ionizing radiations 0.0 Bq C-14 eq
Freshwater eutrophication 0.0 kg PO4 P-lim eq
Fossil and nuclear energy use 0.0 MJ

Parameters

In [23]:
parameters = client.get_parameters(o.ProductSystem, model.id)
for param in parameters:
    print(f"parameter: {param.name} = {param.value}")
param = parameters[0]

parameter: NRJ_Intensivity = 1.0


Adjust the NRJ Intensity


In [25]:

### TESTING ####
A = o.ParameterRedef(name='NRJ_Intensivity', value = 30)
print (A)
for param in parameters:
    print(f"parameter: {param.name} = {param.value}")
param = parameters[0]

ParameterRedef(context=None, description=None, is_protected=None, name='NRJ_Intensivity', uncertainty=None, value=30)
parameter: NRJ_Intensivity = 1.0


In [24]:
#### TESTING ###

A = param.context
print (A)

Ref(id='08e41429-497d-47a0-b09f-7cc75b35d83b', category=None, description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, name='aluminium foil production', process_type=<ProcessType.UNIT_PROCESS: 'UNIT_PROCESS'>, ref_unit=None, ref_type=<RefType.Process: 'Process'>)


In [30]:
for x in range(0, 10, 1):
    setup = o.CalculationSetup(
        target=model,
        impact_method=method,
        parameters=[
            o.ParameterRedef(name='NRJ_Intensivity', value=x,context=param.context)
        ],
    )
    result = client.calculate(setup)
    assert result
    result.wait_until_ready()
    impacts = result.get_total_impacts()
    for i in impacts:
        if i.impact_category.name == "Climate change, long term":
            print(f"{param.name}: {x} => {i.amount : .3f} kg CO2 eq")
    result.dispose()



NRJ_Intensivity: 0 =>  300.000 kg CO2 eq
NRJ_Intensivity: 1 =>  306.000 kg CO2 eq
NRJ_Intensivity: 2 =>  312.000 kg CO2 eq
NRJ_Intensivity: 3 =>  318.000 kg CO2 eq
NRJ_Intensivity: 4 =>  324.000 kg CO2 eq
NRJ_Intensivity: 5 =>  330.000 kg CO2 eq
NRJ_Intensivity: 6 =>  336.000 kg CO2 eq
NRJ_Intensivity: 7 =>  342.000 kg CO2 eq
NRJ_Intensivity: 8 =>  348.000 kg CO2 eq
NRJ_Intensivity: 9 =>  354.000 kg CO2 eq


In [31]:
refs = client.get_descriptors(o.FlowProperty)
for ref in refs:
    print(ref.name)
# Area
# Area*time
# Duration
# Energy

Biotic Production (Transf.)
Length*time
Mass
Items*Length
Gross calorific value
Volume*Length
Mass*time
Vehicle transport
Radioactivity
Biotic Production (Occ.)
Area*time
Groundwater Replenishment (Occ.)
Energy/mass*time
Mechanical Filtration (Occ.)
Duration
Market value, bulk prices
Erosion Resistance (Transf.)
Mechanical Filtration (Transf.)
Net calorific value
Area
Energy/area*time
Physicochemical Filtration (Occ.)
Energy
Erosion Resistance (Occ.)
Normal Volume
Volume*time
Number of items
Volume
Person transport
Groundwater Replenishment (Transf.)
Length
Goods transport (mass*distance)
Physicochemical Filtration (Transf.)
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mass
Energy
Number of items
Mas